In [1]:
!pip install simpledbf

In [2]:
import shutil
import urllib.request as request
from contextlib import closing
import zipfile
import shutil
import sys
import os
from pathlib import Path
import pandas as pd
from simpledbf import Dbf5
import platform

def download(url, save_filepath):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(save_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)

def download_ftp(url, savepath):
    with closing(request.urlopen(url)) as r:
        with open(savepath, 'wb') as f:
            shutil.copyfileobj(r, f)

root = Path(".")

In [3]:
dbc_save_folder = root / "SINASC_DBC"
dbf_save_folder = root / "SINASC_DBF"
csv_save_folder = root / "SINASC_CSV"

dbc_save_folder.mkdir(exist_ok=True)
dbf_save_folder.mkdir(exist_ok=True)
csv_save_folder.mkdir(exist_ok=True)


In [4]:
SINASC_URLS = [
    f"ftp://ftp.datasus.gov.br/dissemin/publicos/SINASC/1996_/Dados/DNRES/DNSP{year}.dbc"
    for year in range(2010, 2019 + 1)
]



for url in SINASC_URLS:
    print("Downloading", url.split("/")[-1])
    save_path = dbc_save_folder / url.split("/")[-1]
    download_ftp(url, save_path)

In [5]:
%load_ext rpy2.ipython

In [6]:
%%R
# Using read.dbc package
# https://pt.linkedin.com/pulse/datasus-conhe%C3%A7a-nova-ferramenta-para-ler-arquivos-dbc-petruzalek

install.packages("read.dbc")
library("read.dbc")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/read.dbc_1.0.5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 20988 bytes (20 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write

In [7]:
!ls SINASC_CSV

DNSP2010.csv  DNSP2012.csv  DNSP2014.csv  DNSP2016.csv
DNSP2011.csv  DNSP2013.csv  DNSP2015.csv


In [8]:
%%R
files = list.files("SINASC_DBC")
for (file in files){
    
    print(file)

    filename = strsplit(file, ".dbc")[1]
    
    dbc_filepath = paste("./SINASC_DBC/", file, sep="")

    csv_filepath = paste("./SINASC_CSV/", filename, ".csv", sep="")
    
    df <- read.dbc(dbc_filepath)

    write.csv(df, csv_filepath)
}

[1] "DNSP2010.dbc"
[1] "DNSP2011.dbc"
[1] "DNSP2012.dbc"
[1] "DNSP2013.dbc"
[1] "DNSP2014.dbc"
[1] "DNSP2015.dbc"
[1] "DNSP2016.dbc"
[1] "DNSP2017.dbc"
[1] "DNSP2018.dbc"
[1] "DNSP2019.dbc"


In [10]:
union_df = None
dfs = []
cols = ["DTNASC", "QTDFILVIVO", "SEXO", "IDADEMAE", "PESO", "GRAVIDEZ", "CONSULTAS", "RACACOR", "CODMUNNASC", "ESTCIVMAE", "ESCMAE", "PARTO", "IDANOMAL"]
union_csv_filename = "union.csv"
for csv_filepath in csv_save_folder.glob("*.csv"):
    if csv_filepath.name == union_csv_filename:
        continue
    df = pd.read_csv(csv_filepath, usecols=cols)
    print(csv_filepath, len(df.columns))
    sorted_cols = list(sorted(df.columns))
    df = df[cols]
    dfs += [df]
    if union_df is None:
        union_df = df
    else:
        union_df = pd.concat([union_df, df])
union_df.to_csv(csv_save_folder / union_csv_filename)

SINASC_CSV/DNSP2013.csv 13
SINASC_CSV/DNSP2010.csv 13
SINASC_CSV/DNSP2014.csv 13
SINASC_CSV/DNSP2016.csv 13
SINASC_CSV/DNSP2018.csv 13
SINASC_CSV/DNSP2017.csv 13
SINASC_CSV/DNSP2015.csv 13
SINASC_CSV/DNSP2012.csv 13
SINASC_CSV/DNSP2011.csv 13
SINASC_CSV/DNSP2019.csv 13


In [12]:
import pandas as pd

pd.read_csv(csv_save_folder / "union.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,DTNASC,QTDFILVIVO,SEXO,IDADEMAE,PESO,GRAVIDEZ,CONSULTAS,RACACOR,CODMUNNASC,ESTCIVMAE,ESCMAE,PARTO,IDANOMAL
0,0,28102013,99.0,1,22.0,9999.0,1.0,9.0,NaN,355030,9.0,1.0,9.0,9.0
1,1,31052013,99.0,2,99.0,9999.0,1.0,9.0,NaN,355030,9.0,9.0,9.0,9.0
2,2,21092013,0.0,1,37.0,3200.0,1.0,9.0,NaN,353070,NaN,5.0,1.0,9.0
3,3,13122013,0.0,1,32.0,2740.0,1.0,4.0,NaN,355620,2.0,5.0,1.0,2.0
4,4,20022013,1.0,1,27.0,3075.0,NaN,NaN,NaN,354410,9.0,3.0,NaN,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6101363,583186,5122019,2.0,2,27.0,3040.0,1.0,4.0,4.0,520870,1.0,4.0,2.0,2.0
6101364,583187,1082019,0.0,2,31.0,3480.0,1.0,4.0,1.0,530010,5.0,5.0,1.0,2.0
6101365,583188,17092019,1.0,1,33.0,2940.0,1.0,3.0,2.0,530010,1.0,5.0,2.0,2.0
6101366,583189,22112019,1.0,2,33.0,2770.0,1.0,4.0,4.0,530010,2.0,5.0,2.0,2.0
